In [1]:
import pandas as pd
import numpy as np
import holidays
from datetime import timedelta
from holidayskr import year_holidays, is_holiday
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# cpi 데이터 로드
df_cpi = pd.read_excel(r"C:\Users\james\J_Data_Lab\Project_KTX\data\korea_cpi_data.xlsx")
df_cpi.head()

,발표일,적용일,시간,실제,예측,이전
0,2024-04-02,3월,08:00:00,0.031,0.031,0.031
1,2024-03-06,2월,08:00:00,0.031,0.029,0.028
2,2024-02-02,1월,08:00:00,0.028,0.029,0.032
3,2023-12-29,12월,08:00:00,0.032,0.033,0.033
4,2023-12-05,11월,08:00:00,0.033,0.037,0.038


In [5]:
# 데이터 불러오기
file_path = r"C:\Users\james\J_Data_Lab\Project_KTX\data\ktx_prep_v1.csv"
df = pd.read_csv(file_path)
df.head()

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,운행요일구분_화,상행_KTX,상행_KTX-산천,상행_KTX-호남,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,False,57.0,0.0,0.0,55.0,0.0,0.0,1,True,False
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,False,0.0,6.0,0.0,0.0,6.0,0.0,1,True,False
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,False,8.0,0.0,0.0,8.0,0.0,0.0,1,True,False
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,False,0.0,13.0,0.0,0.0,13.0,0.0,1,True,False
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,False,1.0,0.0,0.0,1.0,0.0,0.0,1,True,False


In [6]:
def eda_basic_info(df, df_name="DataFrame"):
    print(f"\n===== {df_name} =====")
    print("Head (5 rows):")
    display(df.head())
    
    print("\nInfo:")
    print(df.info())
    
    print("\nDescribe (numeric):")
    display(df.describe())
    
    print("\nNull counts:")
    print(df.isnull().sum())
    
    print("="*40)

In [7]:
eda_basic_info(df, "df")


===== df =====
Head (5 rows):


,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,운행요일구분_화,상행_KTX,상행_KTX-산천,상행_KTX-호남,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,False,57.0,0.0,0.0,55.0,0.0,0.0,1,True,False
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,False,0.0,6.0,0.0,0.0,6.0,0.0,1,True,False
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,False,8.0,0.0,0.0,8.0,0.0,0.0,1,True,False
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,False,0.0,13.0,0.0,0.0,13.0,0.0,1,True,False
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,False,1.0,0.0,0.0,1.0,0.0,0.0,1,True,False



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38416 entries, 0 to 38415
Data columns (total 25 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   운행일자       38416 non-null  object 
 1   주운행선       38416 non-null  int64  
 2   역무열차종      38416 non-null  int64  
 3   공급차량수      38416 non-null  int64  
 4   공급좌석합계수    38416 non-null  int64  
 5   승차수입금액     38416 non-null  int64  
 6   승차인원수      38416 non-null  int64  
 7   승차연인거리     38416 non-null  int64  
 8   좌석거리       38416 non-null  int64  
 9   운행요일구분_금   38416 non-null  bool   
 10  운행요일구분_목   38416 non-null  bool   
 11  운행요일구분_수   38416 non-null  bool   
 12  운행요일구분_월   38416 non-null  bool   
 13  운행요일구분_일   38416 non-null  bool   
 14  운행요일구분_토   38416 non-null  bool   
 15  운행요일구분_화   38416 non-null  bool   
 16  상행_KTX     38416 non-null  float64
 17  상행_KTX-산천  38416 non-null  float64
 18  상행_KTX-호남  38416 non-null  float64
 19  하행_KTX     38416 non-null  float64
 20 

,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,상행_KTX,상행_KTX-산천,상행_KTX-호남,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부
count,38416.000000,38416.000000,38416.000000,38416.000000,3.841600e+04,38416.000000,3.841600e+04,3.841600e+04,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000
mean,2.044955,1.139369,303.709626,15684.878384,4.517961e+08,13754.957153,3.326658e+07,6.182887e+07,6.835407,2.995184,0.860475,6.735267,2.981857,0.862557,0.043836
std,1.551774,1.171995,507.013050,26747.721171,8.529488e+08,24700.056541,6.265189e+07,1.096667e+08,15.143821,4.545374,1.989757,14.786248,4.529586,2.002031,0.204733
min,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,64.000000,3280.000000,1.068817e+08,3406.750000,7.924979e+06,1.355993e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.000000,1.000000,128.000000,6140.000000,1.955657e+08,6422.500000,1.465825e+07,2.542900e+07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,4.000000,3.000000,224.000000,11460.000000,3.232581e+08,10377.250000,2.386877e+07,4.837648e+07,6.000000,6.000000,0.000000,6.000000,6.000000,0.000000,0.000000
max,4.000000,3.000000,2394.000000,124173.000000,5.774399e+09,158416.000000,4.416283e+08,5.225215e+08,116.000000,24.000000,15.000000,114.000000,26.000000,15.000000,1.000000



Null counts:
운행일자         0
주운행선         0
역무열차종        0
공급차량수        0
공급좌석합계수      0
승차수입금액       0
승차인원수        0
승차연인거리       0
좌석거리         0
운행요일구분_금     0
운행요일구분_목     0
운행요일구분_수     0
운행요일구분_월     0
운행요일구분_일     0
운행요일구분_토     0
운행요일구분_화     0
상행_KTX       0
상행_KTX-산천    0
상행_KTX-호남    0
하행_KTX       0
하행_KTX-산천    0
하행_KTX-호남    0
공휴일여부        0
평일           0
주말           0
dtype: int64


In [8]:
eda_basic_info(df_cpi, "cpi")


===== cpi =====
Head (5 rows):


,발표일,적용일,시간,실제,예측,이전
0,2024-04-02,3월,08:00:00,0.031,0.031,0.031
1,2024-03-06,2월,08:00:00,0.031,0.029,0.028
2,2024-02-02,1월,08:00:00,0.028,0.029,0.032
3,2023-12-29,12월,08:00:00,0.032,0.033,0.033
4,2023-12-05,11월,08:00:00,0.033,0.037,0.038



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111 entries, 0 to 110
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   발표일     111 non-null    datetime64[ns]
 1   적용일     111 non-null    object        
 2   시간      110 non-null    object        
 3   실제      110 non-null    float64       
 4   예측      110 non-null    float64       
 5   이전      110 non-null    float64       
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 5.3+ KB
None

Describe (numeric):


,발표일,실제,예측,이전
count,111,110.000000,110.000000,110.000000
mean,2019-09-02 06:42:09.729729792,0.019555,0.019500,0.019345
min,2015-02-03 00:00:00,-0.004000,-0.003000,-0.004000
25%,2017-05-17 00:00:00,0.007000,0.008000,0.007000
50%,2019-09-03 00:00:00,0.015000,0.015500,0.014500
75%,2021-12-16 12:00:00,0.027750,0.028500,0.026750
max,2024-04-02 00:00:00,0.063000,0.063000,0.063000
std,NaN,0.015856,0.015431,0.015856



Null counts:
발표일    0
적용일    0
시간     1
실제     1
예측     1
이전     1
dtype: int64


In [9]:
df_cpi

,발표일,적용일,시간,실제,예측,이전
0,2024-04-02,3월,08:00:00,0.031,0.031,0.031
1,2024-03-06,2월,08:00:00,0.031,0.029,0.028
2,2024-02-02,1월,08:00:00,0.028,0.029,0.032
3,2023-12-29,12월,08:00:00,0.032,0.033,0.033
4,2023-12-05,11월,08:00:00,0.033,0.037,0.038
...,...,...,...,...,...,...
106,2015-06-02,5월,08:00:00,0.005,0.005,0.004
107,2015-05-01,4월,08:00:00,0.004,0.004,0.004
108,2015-04-01,3월,08:00:00,0.004,0.004,0.005
109,2015-03-03,2월,08:00:00,0.005,0.008,0.008


In [10]:
df

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,운행요일구분_화,상행_KTX,상행_KTX-산천,상행_KTX-호남,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,False,57.0,0.0,0.0,55.0,0.0,0.0,1,True,False
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,False,0.0,6.0,0.0,0.0,6.0,0.0,1,True,False
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,False,8.0,0.0,0.0,8.0,0.0,0.0,1,True,False
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,False,0.0,13.0,0.0,0.0,13.0,0.0,1,True,False
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,False,1.0,0.0,0.0,1.0,0.0,0.0,1,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38411,2024-03-31,2,0,108,5730,207982728,7169,14516692,21314645,False,...,False,3.0,0.0,0.0,3.0,0.0,0.0,0,False,True
38412,2024-03-31,2,1,176,8338,348627630,12278,24338423,31091265,False,...,False,0.0,11.0,0.0,0.0,11.0,0.0,0,False,True
38413,2024-03-31,2,3,32,1640,77434004,2571,5292330,6059800,False,...,False,0.0,0.0,2.0,0.0,0.0,2.0,0,False,True
38414,2024-03-31,1,0,216,11460,491723566,15533,36739684,48564615,False,...,False,6.0,0.0,0.0,6.0,0.0,0.0,0,False,True


In [11]:
# 최종적용일 생성
df_cpi['년도'] = pd.to_datetime(df_cpi['발표일']).dt.year  # 발표일에서 연도 추출
df_cpi['월'] = df_cpi['적용일'].str.replace('월', '').astype(int)  # 적용일에서 숫자만 추출

# 최종적용일 열 추가 (YYYY-MM 형식)
df_cpi['최종적용일'] = df_cpi['년도'].astype(str) + '-' + df_cpi['월'].astype(str).str.zfill(2)

# 필요 없는 열 삭제
df_cpi = df_cpi.drop(['년도', '월'], axis=1)

# 결과 출력
df_cpi

,발표일,적용일,시간,실제,예측,이전,최종적용일
0,2024-04-02,3월,08:00:00,0.031,0.031,0.031,2024-03
1,2024-03-06,2월,08:00:00,0.031,0.029,0.028,2024-02
2,2024-02-02,1월,08:00:00,0.028,0.029,0.032,2024-01
3,2023-12-29,12월,08:00:00,0.032,0.033,0.033,2023-12
4,2023-12-05,11월,08:00:00,0.033,0.037,0.038,2023-11
...,...,...,...,...,...,...,...
106,2015-06-02,5월,08:00:00,0.005,0.005,0.004,2015-05
107,2015-05-01,4월,08:00:00,0.004,0.004,0.004,2015-04
108,2015-04-01,3월,08:00:00,0.004,0.004,0.005,2015-03
109,2015-03-03,2월,08:00:00,0.005,0.008,0.008,2015-02


In [13]:
# 마지막 행의 실제값에 이전 행의 '이전' 값 삽입
df_cpi.loc[df_cpi.index[-1], '실제'] = df_cpi.loc[df_cpi.index[-2], '이전']

# 결과 출력
df_cpi

,발표일,적용일,시간,실제,예측,이전,최종적용일
0,2024-04-02,3월,08:00:00,0.031,0.031,0.031,2024-03
1,2024-03-06,2월,08:00:00,0.031,0.029,0.028,2024-02
2,2024-02-02,1월,08:00:00,0.028,0.029,0.032,2024-01
3,2023-12-29,12월,08:00:00,0.032,0.033,0.033,2023-12
4,2023-12-05,11월,08:00:00,0.033,0.037,0.038,2023-11
...,...,...,...,...,...,...,...
106,2015-06-02,5월,08:00:00,0.005,0.005,0.004,2015-05
107,2015-05-01,4월,08:00:00,0.004,0.004,0.004,2015-04
108,2015-04-01,3월,08:00:00,0.004,0.004,0.005,2015-03
109,2015-03-03,2월,08:00:00,0.005,0.008,0.008,2015-02


In [14]:
# 필요 없는 열 삭제
df_cpi = df_cpi.drop(['발표일', '적용일','시간','예측','이전'], axis=1)

In [15]:
df_cpi

,실제,최종적용일
0,0.031,2024-03
1,0.031,2024-02
2,0.028,2024-01
3,0.032,2023-12
4,0.033,2023-11
...,...,...
106,0.005,2015-05
107,0.004,2015-04
108,0.004,2015-03
109,0.005,2015-02


In [16]:
# '운행일자'를 datetime 형식으로 변환
df['운행일자'] = pd.to_datetime(df['운행일자'])

# 년, 월, 일 변수 생성
df['년도'] = df['운행일자'].dt.year
df['월'] = df['운행일자'].dt.month
df['일'] = df['운행일자'].dt.day

# 주차 변수 생성 (ISO 기준, 한 주의 시작을 월요일로 가정)
df['주차'] = df['운행일자'].dt.isocalendar().week

# 결과 출력
df.head()

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말,년도,월,일,주차
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,55.0,0.0,0.0,1,True,False,2015,1,1,1
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,0.0,6.0,0.0,1,True,False,2015,1,1,1
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,8.0,0.0,0.0,1,True,False,2015,1,1,1
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,0.0,13.0,0.0,1,True,False,2015,1,1,1
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,1.0,0.0,0.0,1,True,False,2015,1,1,1


In [17]:
df

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말,년도,월,일,주차
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,55.0,0.0,0.0,1,True,False,2015,1,1,1
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,0.0,6.0,0.0,1,True,False,2015,1,1,1
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,8.0,0.0,0.0,1,True,False,2015,1,1,1
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,0.0,13.0,0.0,1,True,False,2015,1,1,1
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,1.0,0.0,0.0,1,True,False,2015,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38411,2024-03-31,2,0,108,5730,207982728,7169,14516692,21314645,False,...,3.0,0.0,0.0,0,False,True,2024,3,31,13
38412,2024-03-31,2,1,176,8338,348627630,12278,24338423,31091265,False,...,0.0,11.0,0.0,0,False,True,2024,3,31,13
38413,2024-03-31,2,3,32,1640,77434004,2571,5292330,6059800,False,...,0.0,0.0,2.0,0,False,True,2024,3,31,13
38414,2024-03-31,1,0,216,11460,491723566,15533,36739684,48564615,False,...,6.0,0.0,0.0,0,False,True,2024,3,31,13


In [18]:
df_cpi[['년도', '월']] = df_cpi['최종적용일'].str.split('-', expand=True).astype(int)

# 두 데이터프레임 병합
merged_df = pd.merge(df, df_cpi, on=['년도', '월'], how='left')

# 결과 출력
merged_df

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,하행_KTX-호남,공휴일여부,평일,주말,년도,월,일,주차,실제,최종적용일
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38411,2024-03-31,2,0,108,5730,207982728,7169,14516692,21314645,False,...,0.0,0,False,True,2024,3,31,13,0.031,2024-03
38412,2024-03-31,2,1,176,8338,348627630,12278,24338423,31091265,False,...,0.0,0,False,True,2024,3,31,13,0.031,2024-03
38413,2024-03-31,2,3,32,1640,77434004,2571,5292330,6059800,False,...,2.0,0,False,True,2024,3,31,13,0.031,2024-03
38414,2024-03-31,1,0,216,11460,491723566,15533,36739684,48564615,False,...,0.0,0,False,True,2024,3,31,13,0.031,2024-03


In [19]:
eda_basic_info(merged_df, "최종")


===== 최종 =====
Head (5 rows):


,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,하행_KTX-호남,공휴일여부,평일,주말,년도,월,일,주차,실제,최종적용일
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,0.0,1,True,False,2015,1,1,1,0.008,2015-01



Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 38416 entries, 0 to 38415
Data columns (total 31 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   운행일자       38416 non-null  datetime64[ns]
 1   주운행선       38416 non-null  int64         
 2   역무열차종      38416 non-null  int64         
 3   공급차량수      38416 non-null  int64         
 4   공급좌석합계수    38416 non-null  int64         
 5   승차수입금액     38416 non-null  int64         
 6   승차인원수      38416 non-null  int64         
 7   승차연인거리     38416 non-null  int64         
 8   좌석거리       38416 non-null  int64         
 9   운행요일구분_금   38416 non-null  bool          
 10  운행요일구분_목   38416 non-null  bool          
 11  운행요일구분_수   38416 non-null  bool          
 12  운행요일구분_월   38416 non-null  bool          
 13  운행요일구분_일   38416 non-null  bool          
 14  운행요일구분_토   38416 non-null  bool          
 15  운행요일구분_화   38416 non-null  bool          
 16  상행_KTX     38416 non-null  float6

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,상행_KTX,...,상행_KTX-호남,하행_KTX,하행_KTX-산천,하행_KTX-호남,공휴일여부,년도,월,일,주차,실제
count,38416,38416.000000,38416.000000,38416.000000,38416.000000,3.841600e+04,38416.000000,3.841600e+04,3.841600e+04,38416.000000,...,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.000000,38416.0,38416.000000
mean,2019-12-17 06:59:13.519366912,2.044955,1.139369,303.709626,15684.878384,4.517961e+08,13754.957153,3.326658e+07,6.182887e+07,6.835407,...,0.860475,6.735267,2.981857,0.862557,0.043836,2019.466082,6.452259,15.733314,26.303728,0.021976
min,2015-01-01 00:00:00,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2015.000000,1.000000,1.000000,1.0,-0.004000
25%,2017-07-17 00:00:00,1.000000,0.000000,64.000000,3280.000000,1.068817e+08,3406.750000,7.924979e+06,1.355993e+07,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2017.000000,3.000000,8.000000,13.0,0.008000
50%,2020-02-09 00:00:00,2.000000,1.000000,128.000000,6140.000000,1.955657e+08,6422.500000,1.465825e+07,2.542900e+07,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,2020.000000,6.000000,16.000000,26.0,0.018000
75%,2022-06-22 06:00:00,4.000000,3.000000,224.000000,11460.000000,3.232581e+08,10377.250000,2.386877e+07,4.837648e+07,6.000000,...,0.000000,6.000000,6.000000,0.000000,0.000000,2022.000000,9.000000,23.000000,39.0,0.033000
max,2024-03-31 00:00:00,4.000000,3.000000,2394.000000,124173.000000,5.774399e+09,158416.000000,4.416283e+08,5.225215e+08,116.000000,...,15.000000,114.000000,26.000000,15.000000,1.000000,2024.000000,12.000000,31.000000,53.0,0.063000
std,NaN,1.551774,1.171995,507.013050,26747.721171,8.529488e+08,24700.056541,6.265189e+07,1.096667e+08,15.143821,...,1.989757,14.786248,4.529586,2.002031,0.204733,2.754816,3.461216,8.802850,15.109172,0.016711



Null counts:
운행일자         0
주운행선         0
역무열차종        0
공급차량수        0
공급좌석합계수      0
승차수입금액       0
승차인원수        0
승차연인거리       0
좌석거리         0
운행요일구분_금     0
운행요일구분_목     0
운행요일구분_수     0
운행요일구분_월     0
운행요일구분_일     0
운행요일구분_토     0
운행요일구분_화     0
상행_KTX       0
상행_KTX-산천    0
상행_KTX-호남    0
하행_KTX       0
하행_KTX-산천    0
하행_KTX-호남    0
공휴일여부        0
평일           0
주말           0
년도           0
월            0
일            0
주차           0
실제           0
최종적용일        0
dtype: int64


In [20]:
# '실제' 변수명을 'korea_cpi'로 변경
merged_df = merged_df.rename(columns={'실제': 'korea_cpi'})

# '최종적용일' 열 삭제
merged_df = merged_df.drop(columns=['최종적용일'], errors='ignore')

# 결과 출력
merged_df

,운행일자,주운행선,역무열차종,공급차량수,공급좌석합계수,승차수입금액,승차인원수,승차연인거리,좌석거리,운행요일구분_금,...,하행_KTX-산천,하행_KTX-호남,공휴일여부,평일,주말,년도,월,일,주차,korea_cpi
0,2015-01-01,0,0,2016,104152,3725143600,101525,288685921,440665165,False,...,0.0,0.0,1,True,False,2015,1,1,1,0.008
1,2015-01-01,0,1,112,5082,176309700,5142,13328413,19245897,False,...,6.0,0.0,1,True,False,2015,1,1,1,0.008
2,2015-01-01,4,0,288,14776,243662000,8832,22963515,57465746,False,...,0.0,0.0,1,True,False,2015,1,1,1,0.008
3,2015-01-01,4,1,208,9438,279357800,9917,25308662,37074279,False,...,13.0,0.0,1,True,False,2015,1,1,1,0.008
4,2015-01-01,3,0,36,1862,54500600,1879,5035279,7824124,False,...,0.0,0.0,1,True,False,2015,1,1,1,0.008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38411,2024-03-31,2,0,108,5730,207982728,7169,14516692,21314645,False,...,0.0,0.0,0,False,True,2024,3,31,13,0.031
38412,2024-03-31,2,1,176,8338,348627630,12278,24338423,31091265,False,...,11.0,0.0,0,False,True,2024,3,31,13,0.031
38413,2024-03-31,2,3,32,1640,77434004,2571,5292330,6059800,False,...,0.0,2.0,0,False,True,2024,3,31,13,0.031
38414,2024-03-31,1,0,216,11460,491723566,15533,36739684,48564615,False,...,0.0,0.0,0,False,True,2024,3,31,13,0.031


In [31]:
# 데이터 저장
merged_df.to_csv( r"C:\Users\james\J_Data_Lab\Project_KTX\data\ktx_prep_v2.csv", index=False)